In [40]:
from sklearn import metrics
import numpy as np
import pandas as pd
import pickle
import re
import mymodule

In [29]:
with open('model/lda_model.pkl', 'rb') as f:
  lda_model = pickle.load(f) #import lda model
  
data = pd.read_csv('dataset/DatasetLegal.csv') #read data
question_data = data['question']
train_question_data = question_data[:6031]
test_question_data = question_data[6031:]

with open('model/topic_dict.pkl', 'rb') as f:
  topic_dict = pickle.load(f)

with open('model/question_lda.pkl', 'rb') as f:
   question_lda = pickle.load(f)

In [9]:
str_answer = train_question_data.astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,\.!?#]', '', x))
corpus = [mymodule.preprocess(text) for text in str_answer]
pred_score = [lda_model[text] for text in corpus]

In [30]:
pred_score

[1, 3, 0, 6, 0, 4, 0, 4, 6, 4]
['family', 'children', 'contract', 'succession', 'contract', 'sentence', 'contract', 'sentence', 'succession', 'sentence']


In [31]:
# for each_doc in pred_score:
#     i = 0
#     for each_topic in each_doc:
#         if(each_topic[0] == i): i+=1
#         else: 
#             each_doc.insert(i, (i, 0))
#             i+=1
# จะติดว่า ถ้าตัวสุดท้ายหายไป มันจะไม่สน เช่น index 5 หาย
# แต่ถ้าเราสร้าง if เช็คแค่ index สุดท้าย ก็จะมีปัญหาเรื่อง ถ้าหาย index 4 5 หรือว่ามากกว่า T^T

0         family
1         family
2       contract
3         family
4       contract
          ...   
7026    contract
7027    contract
7028    criminal
7029    criminal
7030    contract
Name: Classes, Length: 7031, dtype: object

In [34]:
# for each_doc in pred_score:
#     for i in range(6):
#         if(each_doc[i][0] != i): each_doc.insert(i, (i, 0))
# list index out of range 

In [35]:
for each_doc in pred_score:
    i = 0
    for each_topic in each_doc:
        if(each_topic[0] == i): i+=1
        else: 
            each_doc.insert(i, (i, 0))
            i+=1
    while(i < 6):
        each_doc.append((i, 0))
        i+=1

0.4667899303086332

In [38]:
pred_score[6029:]

array([[   0,    1,   19,    0,   32,    4,    5,  317,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,   94, 2044,    0,    8,   33,  149,  158,   48],
       [   0,    6,    8,    0,   14,    7,   32,  987,    2],
       [   0,  538,   34,    0,  464,    5,   11,   67,  995],
       [   0,    2,   14,    0,    4,  774,  145,    5,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int64)

In [41]:
y_predict=[]
for each_topic in pred_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>0.167): temp_pred.append(1)
    else: temp_pred.append(0)
  y_predict.append(temp_pred)
# y_predict = np.array(y_predict)
y_predict

c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.        , 0.        , 0.96460595, 0.        , 0.88888889,
        0.94046173, 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.80662983, 0.        , 0.21948912,
        0.81904762, 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.87857296, 0.        , 0.35204856,
        0.87556561, 0.        , 0.        , 0.        ]),
 array([ 382,    0, 2534, 1056, 2114,  945,    0,    0,    0], dtype=int64))

In [49]:
for each_doc in y_predict:
    each_doc.pop(0)
y_predict

Accuracy: 0.4667899303086332
Precision: 0.7413111803844005
Recall: 0.4667899303086332
F1-score: 0.5401712480059311


c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test = data['Classes']
test = test[:6031]
y_test = []
topics_of_interest = ['Violation', 'family', 'criminal', 'contract', 'labor']
for sample in test:
    temp_test = []
    # Loop over each topic of interest
    for topic in topics_of_interest:
        if topic in sample:
            temp_test.append(1)
        else:
            temp_test.append(0)
    y_test.append(temp_test)

In [ ]:
print(len(y_test))
y_test


In [ ]:
y_true = y_test
y_pred = y_predict

In [ ]:
metrics.accuracy_score(y_true, y_pred)

In [ ]:
metrics.multilabel_confusion_matrix(y_true, y_pred)

In [ ]:
label_names = ['violation', 'family', 'criminal', 'contract', 'labor']
print(metrics.classification_report(y_true, y_pred,target_names=label_names))